In [32]:
import numpy as np
import pandas as pd
from scipy.linalg import hadamard

In [33]:
#Воздействует на исходное состояние before_impulse импульсом impulse, base - колчиство спинов
def preparational_impulse(before_impulse, impulse ,base = 3):
    sign = 1
    after_impulse=""
    #в конце импульса хранится количество пиков, попавших 
    if(len(before_impulse)!=base):
        #отделяем число
        sign = sign/float(before_impulse[base::])
        before_impulse = before_impulse[:base:]
    
    if(len(before_impulse)==base):
        for i in range(base):
            if(before_impulse[i]=='0'):
                after_impulse+='0'
            
            if(before_impulse[i]=='1'):
                if(impulse[i]=='0'):
                    after_impulse+='1'
                elif(impulse[i]=='1'):
                    after_impulse+='1'
                elif(impulse[i]=='2'):
                    after_impulse+='3'
            
            if(before_impulse[i]=='2'):
                if(impulse[i]=='0'):
                    after_impulse+='2'
                elif(impulse[i]=='1'):
                    after_impulse+='3'
                    sign*=-1
                elif(impulse[i]=='2'):
                    after_impulse+='2'
            
            
            if(before_impulse[i]=='3'):
                if(impulse[i]=='0'):
                    after_impulse+='3'
                elif(impulse[i]=='1'):
                    after_impulse+='2'
                elif(impulse[i]=='2'):
                    after_impulse+='1'
                    sign*=-1
    return after_impulse, sign

In [34]:
#Дозаполняет словарь density_dict значениями различных rho_i. В before_impulse подаются состояния в нужном порядке(см как в статье сверху вних).
#raw_data - DataFrame, где содержатся колонки impulse, и все before_impulse 
def find_density_elements(density_dict, before_impulses, raw_data, num_of_spins):
    #Вот здесь медленно сделано!!!!!!!!!!!
    for index, row in raw_data.iterrows():
        
        
        final_impulses=[]
        prep_impulse = row['impulses']
        #ЗДЕСЬ НАДО ИСПРАВИТЬ ЧТОБЫ ПОРЯДОК БЫЛ НОРМАЛЬНЫЙ НЕСМОТРЯ НА ТО КАК РАСПОЛОЖИЛИСЬ в датафрейме
        #UPD вроде все ок и мы это проследили
        for before_impulse in before_impulses:
            final_impulses.append(preparational_impulse(before_impulse, prep_impulse, 3))
        
        
        
        H = hadamard(len(before_impulses))
        S=[]
        skip =False
        for impulse in before_impulses:
            if(np.isnan(row[impulse])):
                skip = True
            S.append(row[impulse])
        if(skip):
            print("Continue")
            continue
        S = np.array(S).reshape(-1,1)
        
        rho = np.matmul(H, S)/(2**(num_of_spins-1))
        rho_flat = rho.flatten()
        
        for k in range(len(before_impulses)):
            if(final_impulses[k][0] in density_dict):
                
                density_dict[final_impulses[k][0]]=np.append(density_dict[final_impulses[k][0]],final_impulses[k][1]*rho_flat[k])
                
            else:
                density_dict[final_impulses[k][0]] = []
                density_dict[final_impulses[k][0]].append(final_impulses[k][1]*rho_flat[k])
    
    for key in density_dict:
        density_dict[key]=np.sum(density_dict[key])/np.size(density_dict[key])


In [35]:
def find_prep_impulses3():
    base = 3
    list_of_prep_impulses = []
    for i in range(27):
        k=i
        my_str = ""
        while(k>0):
            my_str+=str(k%base)
            k=k//base
        while(len(my_str)<3):
            my_str+='0'
        my_str = my_str[::-1]
        list_of_prep_impulses.append(my_str)
    return list_of_prep_impulses

In [36]:
def load_file(file_name, num_of_spins, isreal):
    df = pd.read_csv(file_name, index_col=False)
    columns = df.columns
    for col in columns:
        col_list = list(col)
        for i in range(num_of_spins):
            if(not(isreal) and (col_list[i]=='1')):
                col_list[i]='2'
                
            if(col_list[i]=='-'):
                col_list[i] = '0'
            elif(col_list[i]=='+'):
                col_list[i]='3'
        col_str_fin = "".join(col_list)
        df.rename(columns = {col:col_str_fin},inplace = True)
    return df


In [37]:
density_dict={}
num_of_spins =3
#ПОКА ЧТО НАДО заранее указывать импульсы которые удвоенные в наших массивах
#И когда заполняем тоже надо
#TODO Подумать и исправить это как-то
before_impulses_H_real = ["1002", "1302", "1032", "1332"]


list_of_prep_impulses = ["100", "001", "002", "011", "012", "021", "022", "101", "102", "111", 
                                "112", "121", "122", "201", "202", "211", "212", "221", "222", 
                                "000", "010", "020", "100", "110", "120", "200", "210", "220" ]

#По порядку 

raw_data = pd.DataFrame()
raw_data["impulses"] = list_of_prep_impulses
H_data_real = load_file("H_real.csv",3, True)
raw_data = raw_data.join(H_data_real)
raw_data

,impulses,1002,1032,1302,1332
0,100,3.466673,3.466609,3.564338,3.557332
1,001,0.000923,0.001554,-0.001385,-0.000533
2,002,0.006463,0.005407,0.003709,0.004479
3,011,0.000163,0.001853,-0.001812,-0.002227
4,012,-0.003393,-0.004111,0.001303,0.000968
5,021,0.001416,0.002408,0.001318,0.000948
6,022,0.000976,0.000460,0.003705,0.003573
7,101,3.376267,3.356269,3.478077,3.461919
8,102,3.412784,3.399656,3.493748,3.481247
9,111,3.454551,3.454490,3.536511,3.527990


In [38]:
find_density_elements(density_dict, before_impulses_H_real, raw_data, num_of_spins)
density_dict

{'100': 0.9071584795299955,
 '130': -0.014081773657223054,
 '103': 0.0017499441051310064,
 '133': 0.00021693150371009616,
 '102': 0.0013782830092606976,
 '132': 0.00025382576715312364,
 '101': -0.0016280521872134833,
 '131': -0.00015335973156946383,
 '120': -0.011243618364837386,
 '122': -0.000660357862893339,
 '121': -0.00012949630298318915,
 '110': 0.012080175502232275,
 '112': -7.744929845072714e-05,
 '111': 0.00010927837337242681,
 '300': 0.02011867717621952,
 '330': 0.4886114558909031,
 '302': 0.00797663223025461,
 '332': -0.023622682317425564,
 '301': -0.006151737745386965,
 '331': 0.024957680351412465,
 '320': 0.488615792885954,
 '322': -0.0241087765757938,
 '321': 0.023538246223981457,
 '310': -0.48575676689102404,
 '312': 0.021374141168691907,
 '311': -0.023897068207729383,
 '123': -0.00018696349623196422,
 '113': -0.000329066480421618,
 '303': -0.0012368195260903613,
 '333': -0.027821331923354342,
 '323': -0.029293695765551298,
 '313': 0.026260714456087464}

C

In [39]:
before_impulses_C_real = ["010", "310", "013", "313"]
C_data_real = load_file("C_real.csv",3, True)
raw_data = raw_data.join(C_data_real)
raw_data

,impulses,1002,1032,1302,1332,010,013,310,313
0,100,3.466673,3.466609,3.564338,3.557332,0.425944,0.311680,0.312807,0.473683
1,001,0.000923,0.001554,-0.001385,-0.000533,0.009767,0.026443,-0.008022,0.016566
2,002,0.006463,0.005407,0.003709,0.004479,-0.016122,-0.027282,-0.007831,0.006794
3,011,0.000163,0.001853,-0.001812,-0.002227,0.403914,0.332112,0.335670,0.450214
4,012,-0.003393,-0.004111,0.001303,0.000968,0.370027,0.280095,0.285973,0.396971
5,021,0.001416,0.002408,0.001318,0.000948,0.282729,-0.240846,0.252767,-0.303885
6,022,0.000976,0.000460,0.003705,0.003573,0.305730,-0.214192,0.274656,-0.300182
7,101,3.376267,3.356269,3.478077,3.461919,-0.005009,-0.000629,0.014395,-0.024089
8,102,3.412784,3.399656,3.493748,3.481247,-0.011132,-0.020967,-0.017944,-0.015388
9,111,3.454551,3.454490,3.536511,3.527990,0.426854,0.326289,0.320595,0.450398


In [40]:
find_density_elements(density_dict, before_impulses_C_real, raw_data, num_of_spins)
density_dict

{'100': 0.9071584795299955,
 '130': -0.02469494609961864,
 '103': 0.0017499441051310064,
 '133': 0.008653756554215896,
 '102': 0.0013782830092606976,
 '132': 0.0068766425166099125,
 '101': -0.0016280521872134833,
 '131': -0.004678841708529547,
 '120': -0.011243618364837386,
 '122': -0.000660357862893339,
 '121': -0.00012949630298318915,
 '110': 0.005914362983627707,
 '112': -0.019216373819034734,
 '111': 0.019607545710215616,
 '300': 0.02011867717621952,
 '330': 0.1441268967027695,
 '302': 0.00797663223025461,
 '332': -0.015946002587124274,
 '301': -0.006151737745386965,
 '331': 0.019343249900458208,
 '320': 0.488615792885954,
 '322': -0.0241087765757938,
 '321': 0.023538246223981457,
 '310': -0.07300604527050378,
 '312': 0.02331284012360575,
 '311': -0.02685864161869747,
 '123': -0.00018696349623196422,
 '113': -0.01878466680390583,
 '303': -0.0012368195260903613,
 '333': -0.017077402497214515,
 '323': -0.029293695765551298,
 '313': 0.023638745196720212,
 '010': 0.1880413598334948,
 '

In [41]:
before_impulses_N_real = ["0012", "3012", "0312", "3312"]
N_data_real = load_file("N_real.csv",3, True )
raw_data = raw_data.join(N_data_real)
raw_data

,impulses,1002,1032,1302,1332,010,013,310,313,0012,0312,3012,3312
0,100,3.466673,3.466609,3.564338,3.557332,0.425944,0.311680,0.312807,0.473683,0.140344,0.140174,0.219075,0.218856
1,001,0.000923,0.001554,-0.001385,-0.000533,0.009767,0.026443,-0.008022,0.016566,0.170814,0.170007,0.241463,0.240738
2,002,0.006463,0.005407,0.003709,0.004479,-0.016122,-0.027282,-0.007831,0.006794,0.107196,0.108525,-0.016849,-0.016131
3,011,0.000163,0.001853,-0.001812,-0.002227,0.403914,0.332112,0.335670,0.450214,0.174316,0.172410,0.197234,0.195256
4,012,-0.003393,-0.004111,0.001303,0.000968,0.370027,0.280095,0.285973,0.396971,0.125339,0.124984,-0.053572,-0.054505
5,021,0.001416,0.002408,0.001318,0.000948,0.282729,-0.240846,0.252767,-0.303885,0.156853,0.157172,0.223060,0.223286
6,022,0.000976,0.000460,0.003705,0.003573,0.305730,-0.214192,0.274656,-0.300182,0.151889,0.152102,-0.056100,-0.056272
7,101,3.376267,3.356269,3.478077,3.461919,-0.005009,-0.000629,0.014395,-0.024089,0.123029,0.122870,0.197763,0.197647
8,102,3.412784,3.399656,3.493748,3.481247,-0.011132,-0.020967,-0.017944,-0.015388,0.094973,0.094487,0.038238,0.037320
9,111,3.454551,3.454490,3.536511,3.527990,0.426854,0.326289,0.320595,0.450398,0.165922,0.165201,0.192088,0.191552


In [42]:
find_density_elements(density_dict, before_impulses_N_real, raw_data, num_of_spins)
density_dict

{'100': 0.9071584795299955,
 '130': -0.02469494609961864,
 '103': -0.03555265684817394,
 '133': 0.004363796581972224,
 '102': 0.0013782830092606976,
 '132': 0.0068766425166099125,
 '101': 0.007430985140068373,
 '131': -0.0015710109679597454,
 '120': -0.011243618364837386,
 '122': -0.000660357862893339,
 '121': -4.636029309168432e-05,
 '110': 0.005914362983627707,
 '112': -0.019216373819034734,
 '111': 0.006558623590417684,
 '300': 0.02011867717621952,
 '330': 0.1441268967027695,
 '302': 0.00797663223025461,
 '332': -0.015946002587124274,
 '301': -0.005753767298903875,
 '331': 0.0064516126999499155,
 '320': 0.488615792885954,
 '322': -0.0241087765757938,
 '321': 0.007841945661808429,
 '310': -0.07300604527050378,
 '312': 0.02331284012360575,
 '311': -0.008956034316590466,
 '123': -8.051920383854644e-05,
 '113': -0.009436791079537928,
 '303': 0.031673999460368474,
 '333': -0.008576906979222632,
 '323': -0.014682920087651635,
 '313': 0.011843445883961981,
 '010': 0.1880413598334948,
 '210

In [43]:
before_impulses_H_im = ["2002", "2302", "2032", "2332"]
H_data_im = load_file("H_im.csv",3, False)
raw_data = raw_data.join(H_data_im)
raw_data

find_density_elements(density_dict, before_impulses_H_im, raw_data, num_of_spins)
density_dict


{'100': 0.9071584795299955,
 '130': -0.02469494609961864,
 '103': -0.03555265684817394,
 '133': 0.004363796581972224,
 '102': 0.0013782830092606976,
 '132': 0.0068766425166099125,
 '101': 0.007430985140068373,
 '131': -0.0015710109679597454,
 '120': -0.011243618364837386,
 '122': -0.000660357862893339,
 '121': -4.636029309168432e-05,
 '110': 0.005914362983627707,
 '112': -0.019216373819034734,
 '111': 0.006558623590417684,
 '300': -0.062391634703243354,
 '330': 0.5090936202688516,
 '302': 0.0029817386586707038,
 '332': -0.015870340960741896,
 '301': -0.002148187381042015,
 '331': 0.011070424155314474,
 '320': 0.5710621920240322,
 '322': -0.019325076836439525,
 '321': 0.011269335785340437,
 '310': -0.46799195445296005,
 '312': 0.01859884678865873,
 '311': -0.012117232878373817,
 '123': -8.051920383854644e-05,
 '113': -0.009436791079537928,
 '303': -0.0005043918002821113,
 '333': -0.014445658751633944,
 '323': -0.01511310829885872,
 '313': 0.013739378163545125,
 '010': 0.1880413598334948

In [44]:
before_impulses_C_im = ["020", "320", "023", "323"]
C_data_im = load_file("C_im.csv",3, False)
raw_data = raw_data.join(C_data_im)
raw_data

find_density_elements(density_dict, before_impulses_C_im, raw_data, num_of_spins)
density_dict

{'100': 0.9071584795299955,
 '130': 0.0005893793796945797,
 '103': -0.03555265684817394,
 '133': -0.007580535516046573,
 '102': 0.0013782830092606976,
 '132': 0.005170026327260749,
 '101': 0.007430985140068373,
 '131': 0.009312880879760187,
 '120': 0.0018860590534825745,
 '122': -0.02750628908921551,
 '121': 0.02303212097980235,
 '110': 0.005914362983627707,
 '112': -0.019216373819034734,
 '111': 0.006558623590417684,
 '300': -0.062391634703243354,
 '330': 0.11210312226464907,
 '302': 0.0029817386586707038,
 '332': -0.005118552131184234,
 '301': -0.002148187381042015,
 '331': -0.008617108552111504,
 '320': 0.08130181217999718,
 '322': 0.01772582822049393,
 '321': -0.02388841683722678,
 '310': -0.46799195445296005,
 '312': 0.01859884678865873,
 '311': -0.012117232878373817,
 '123': -0.027480459436172288,
 '113': -0.009436791079537928,
 '303': -0.0005043918002821113,
 '333': 0.0026077203703934416,
 '323': 0.016408861660609512,
 '313': 0.013739378163545125,
 '010': 0.1880413598334948,
 '2

In [45]:
before_impulses_N_im = ["0022", "3022", "0322", "3322"]
N_data_im = load_file("N_im.csv",3, False)
raw_data = raw_data.join(N_data_im)
raw_data

find_density_elements(density_dict, before_impulses_N_im, raw_data, num_of_spins)
density_dict

{'100': 0.9071584795299955,
 '130': 0.0005893793796945797,
 '103': -0.058950297260669166,
 '133': -0.004219085020939265,
 '102': 0.003046148766465165,
 '132': 0.0018134105422552085,
 '101': 0.007430985140068373,
 '131': 0.009312880879760187,
 '120': 0.0018860590534825745,
 '122': -0.008969951078590563,
 '121': 0.02303212097980235,
 '110': 0.005914362983627707,
 '112': -0.006442716971749466,
 '111': 0.006558623590417684,
 '300': -0.062391634703243354,
 '330': 0.11210312226464907,
 '302': -0.012865531575590268,
 '332': -0.0020377955744981277,
 '301': -0.002148187381042015,
 '331': -0.008617108552111504,
 '320': 0.08130181217999718,
 '322': 0.005741118709108571,
 '321': -0.02388841683722678,
 '310': -0.46799195445296005,
 '312': 0.006519903614924715,
 '311': -0.012117232878373817,
 '123': -0.014285966865668739,
 '113': -0.003951875721897704,
 '303': 0.04419646299910682,
 '333': 0.001975860779418017,
 '323': 0.008763117477493693,
 '313': 0.006400385974384368,
 '010': 0.1880413598334948,
 '

In [46]:
len(density_dict)

63

In [35]:
def find_prep_impulses_try():
    base = 4
    list_of_prep_impulses = []
    for i in range(64):
        k=i
        my_str = ""
        while(k>0):
            my_str+=str(k%base)
            k=k//base
        while(len(my_str)<3):
            my_str+='0'
        my_str = my_str[::-1]
        list_of_prep_impulses.append(my_str)
    return list_of_prep_impulses
all_imp =find_prep_impulses_try()

In [36]:
for i in range(64):
    if(not(all_imp[i] in density_dict)):
        print(all_imp[i])

000
002
020
022
200
202
220
222


In [37]:
for(i in range)

SyntaxError: invalid syntax (1908931606.py, line 1)